# **Предобработка данных**

In [656]:
import numpy as np
import pandas as pd

In [657]:
data = pd.read_csv('https://raw.githubusercontent.com/tisheedesh/Project/main/data.csv').drop(['Unnamed: 0'], axis = 1)

<ipython-input-657-b8dad5b8013a>:1: DtypeWarning: Columns (118) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('https://raw.githubusercontent.com/tisheedesh/Project/main/data.csv').drop(['Unnamed: 0'], axis = 1)


> Исследуемый датасет - информация о кольцах с сайта ювелирной компании [SOKOLOV](https://sokolov.ru/), которые мы получили при помощи парсинга. Сам парсинг в файле "название файла с парсингом"



In [658]:
data.head()

,Артикул,Бренд,Для кого,Примерный вес,Тип металла,Проба,Покрытие,Тип вставки,Форма вставки,Количество,...,Чистота 6,Цветность 7,Чистота 7,Огранка 5,Цветность 8,Чистота 8,Цветность 9,Чистота 9,Цветность 10,Чистота 10
0,1012159-3,SOKOLOV,Для женщин,1.66 г,Белое золото,585.0,Родирование,Бриллиант (природный),Круг,6 шт,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1011493,SOKOLOV,Для женщин,1.35 г,Белое золото,585.0,Родирование,Бриллиант (природный),Круг,1 шт,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1012584-3,SOKOLOV,Для женщин,0.9 г,Белое золото,585.0,Родирование,Бриллиант (природный),Круг,23 шт,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1011116,SOKOLOV,Для женщин,2.28 г,Белое золото,585.0,NaN,Бриллиант (природный),Круг,1 шт,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1011545-3,SOKOLOV,Для женщин,1.31 г,Белое золото,585.0,Родирование,Бриллиант (природный),Круг,57 шт,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## **Описание признаков**

In [659]:
print(f'Названия колонок: \n {np.array(data.columns[:32])}')

Названия колонок: 
 ['Артикул' 'Бренд' 'Для кого' 'Примерный вес' 'Тип металла' 'Проба'
 'Покрытие' 'Тип вставки' 'Форма вставки' 'Количество' 'Цвет' 'Вес'
 'Цветность' 'Чистота' 'Огранка' 'Тип вставки 2' 'Форма вставки 2'
 'Количество 2' 'Цвет 2' 'Вес 2' 'Цветность 2' 'Чистота 2' 'Огранка 2'
 'Цена со скидкой' 'Цена без скидки' 'Номер страницы' 'Ширина кольца'
 'Ширина' 'Высота' 'Длина' 'Размеры вставки' 'Коллекция']


В датасете 124 колонки.

Для каждого кольца есть характеристики:

* `Артикул` - артикул кольца на сайте.
* `Бренд` - бренд кольца (здесь - у всех SOKOLOV)
* `Для кого` - для кого предназначено кольцо (пол, возраст)
* `Примерный вес` - вес кольца в граммах
* `Тип металла` - из какого металла сделано кольцо
* `Проба` -
* `Покрытие` -
* `Номер страницы` -
* `Ширина кольца` -
* `Ширина` -
* `Высота` -
* `Длина` -
* `Коллекция` -
* `Номер страницы` -
* `Цена со скидкой` -
* `Цена без скидки`

У каждого кольца есть от одной до 14 вставок драгоценных камней. Мы вынесли все 14 типов вставок как отдельные признаки (столбцы `Тип вставки`), причем у каждой вставки есть свои признаки:
* `Форма вставки` - какой формы вставка драгоценного(-ых) камня(-ей)
* `Количество` - количество камней в данной вставке
* `Цвет` -
* `Вес` -
* `Цветность` -
* `Чистота` -
* `Огранка` -
* `Размеры вставки` -






>### Шаг 1. Проверка дубликатов

Проверяем наличие дубликатов: полных дубликатов нет.

In [660]:
data.duplicated().sum()

0

Однако количество уникальных артикулов не совпадает с количеством колец, что странно, ведь артикул у каждого кольца один.

In [661]:
print(f'Кол-во уникальных артикулов: {data.Артикул.nunique()} \nКол-во колец: {data.shape[0]}')

Кол-во уникальных артикулов: 10209 
Кол-во колец: 10216


Посмотрим, в чем различия колец с одинаковыми артикулами.

In [662]:
articles = data[data['Артикул'].duplicated()]['Артикул'].values
indexes = np.array([])
for elem in articles:
    display(data.loc[data['Артикул'] == elem])
    indexes = np.append(indexes, data.loc[data['Артикул'] == elem].index)

,Артикул,Бренд,Для кого,Примерный вес,Тип металла,Проба,Покрытие,Тип вставки,Форма вставки,Количество,...,Чистота 6,Цветность 7,Чистота 7,Огранка 5,Цветность 8,Чистота 8,Цветность 9,Чистота 9,Цветность 10,Чистота 10
960,716327,SOKOLOV,Для женщин,1.51 г,Красное золото,585.0,Родирование,Корунд,Груша,1 шт,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
961,716327,SOKOLOV,Для женщин,1.51 г,Красное золото,585.0,Родирование,Корунд,Груша,1 шт,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Артикул,Бренд,Для кого,Примерный вес,Тип металла,Проба,Покрытие,Тип вставки,Форма вставки,Количество,...,Чистота 6,Цветность 7,Чистота 7,Огранка 5,Цветность 8,Чистота 8,Цветность 9,Чистота 9,Цветность 10,Чистота 10
1603,716335,SOKOLOV,Для женщин,2.67 г,Красное золото,585.0,NaN,Празиолит,Октагон,1 шт,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1604,716335,SOKOLOV,Для женщин,2.67 г,Красное золото,585.0,NaN,Празиолит,Октагон,1 шт,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Артикул,Бренд,Для кого,Примерный вес,Тип металла,Проба,Покрытие,Тип вставки,Форма вставки,Количество,...,Чистота 6,Цветность 7,Чистота 7,Огранка 5,Цветность 8,Чистота 8,Цветность 9,Чистота 9,Цветность 10,Чистота 10
1890,1012013,SOKOLOV,Для женщин,2.04 г,Комбинированное золото,585.0,Родирование,Бриллиант (природный),Круг,21 шт,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1891,1012013,SOKOLOV,Для женщин,2.04 г,Комбинированное золото,585.0,Родирование,Бриллиант (природный),Круг,21 шт,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Артикул,Бренд,Для кого,Примерный вес,Тип металла,Проба,Покрытие,Тип вставки,Форма вставки,Количество,...,Чистота 6,Цветность 7,Чистота 7,Огранка 5,Цветность 8,Чистота 8,Цветность 9,Чистота 9,Цветность 10,Чистота 10
2231,94011336,SOKOLOV,Для женщин,1.72 г,Родированное серебро,925.0,Эмаль,Фианит,Круг,16 шт,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2232,94011336,SOKOLOV,Для женщин,1.72 г,Родированное серебро,925.0,Эмаль,Фианит,Круг,16 шт,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Артикул,Бренд,Для кого,Примерный вес,Тип металла,Проба,Покрытие,Тип вставки,Форма вставки,Количество,...,Чистота 6,Цветность 7,Чистота 7,Огранка 5,Цветность 8,Чистота 8,Цветность 9,Чистота 9,Цветность 10,Чистота 10
2374,94013377,SOKOLOV,Для женщин,1.29 г,Родированное серебро,925.0,NaN,Фианит,Круг,24 шт,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2375,94013377,SOKOLOV,Для женщин,1.29 г,Родированное серебро,925.0,NaN,Фианит,Круг,24 шт,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Артикул,Бренд,Для кого,Примерный вес,Тип металла,Проба,Покрытие,Тип вставки,Форма вставки,Количество,...,Чистота 6,Цветность 7,Чистота 7,Огранка 5,Цветность 8,Чистота 8,Цветность 9,Чистота 9,Цветность 10,Чистота 10
3159,716001,SOKOLOV,Для женщин,3.15 г,Красное золото,585.0,Родирование,Раухтопаз,Груша,1 шт,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3160,716001,SOKOLOV,Для женщин,3.15 г,Красное золото,585.0,Родирование,Раухтопаз,Груша,1 шт,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Артикул,Бренд,Для кого,Примерный вес,Тип металла,Проба,Покрытие,Тип вставки,Форма вставки,Количество,...,Чистота 6,Цветность 7,Чистота 7,Огранка 5,Цветность 8,Чистота 8,Цветность 9,Чистота 9,Цветность 10,Чистота 10
829,017107,SOKOLOV,Для женщин,2.5 г,Красное золото,585.0,"Алмазная грань, Родирование",Без вставок,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7078,017107,SOKOLOV,Для женщин,2.5 г,Красное золото,585.0,"Алмазная грань, Родирование",Без вставок,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [663]:
for elem in articles:
    dubl = data.loc[data['Артикул'] == elem]
    display(dubl.iloc[0].compare(dubl.iloc[1]))

,self,other
Номер страницы,14,15


,self,other
Номер страницы,23,24


,self,other
Номер страницы,27,28


,self,other
Номер страницы,34,35


,self,other
Номер страницы,36,37


,self,other
Номер страницы,47,48


,self,other
Номер страницы,12,102


Каждый артикул повторяется только два раза, и различия лишь в номере страниц: возможно, во время парсинга кольцо переместилось с одной страницы на другую. Удаляем дубликаты.

In [664]:
for i in indexes[1::2]:
    data.drop(index = i, inplace=True)
data.reset_index(inplace=True)

> ### Шаг 2: работа с пропусками

В данном датасете очень много пропусков. Посмотрим на первые 32 колонки: есть какие-то отдельные признаки с большим количеством пропусков, с ними разберемся позже.

In [665]:
# ну тут норм по пропускам кроме нескольких типа огранки
data.isna().sum()[:32]

index                  0
Артикул                0
Бренд                  0
Для кого              27
Примерный вес          9
Тип металла            0
Проба                 44
Покрытие            3519
Тип вставки            5
Форма вставки       1111
Количество           983
Цвет                1123
Вес                  987
Цветность           6753
Чистота             6747
Огранка             9862
Тип вставки 2       5086
Форма вставки 2     5194
Количество 2        5086
Цвет 2              5211
Вес 2               5088
Цветность 2         8756
Чистота 2           8739
Огранка 2          10065
Цена со скидкой        0
Цена без скидки        0
Номер страницы         0
Ширина кольца       3939
Ширина              5089
Высота              5104
Длина               7133
Размеры вставки     4071
dtype: int64

В последних колонках пропуски -  почти все строки. Так получилось из-за того, что мало колец с большим количеством вставок - их можно считать выбросами. Поэтому целесообразно не рассматривать кольца, в которых вставок более 3 (уже в колонке `Тип вставки 3` пропусков 8497).

In [666]:
data.isna().sum().tail(20)

Размеры вставки 13    10206
Тип вставки 14        10207
Форма вставки 14      10207
Количество 14         10207
Цвет 14               10207
Вес 14                10207
Размеры вставки 14    10207
Цветность 5           10192
Чистота 5             10192
Цветность 6           10203
Чистота 6             10202
Цветность 7           10203
Чистота 7             10203
Огранка 5             10208
Цветность 8           10207
Чистота 8             10207
Цветность 9           10208
Чистота 9             10208
Цветность 10          10208
Чистота 10            10208
dtype: int64

In [667]:
nan_vst_3 = data.isna().sum()['Тип вставки 3']
print(f'Колец с количеством вставок не более двух {nan_vst_3} штуки')

Колец с количеством вставок не более двух 8497 штуки


Убираем строки, где `Тип вставки 3` не является `nan`.

In [668]:
data.drop(data[~data['Тип вставки 3'].isna()].index, inplace=True)
data.reset_index(inplace=True)

Также убираем столбцы, которые отвечают за типы вставок с более чем вторым порядком и за их признаки.

In [669]:
for title in data.columns:
    if title[-1].isnumeric() and title.split()[-1] != '2':
        data.drop([title], axis=1, inplace=True)

In [670]:
data = data[['Артикул', 'Бренд', 'Коллекция', 'Для кого', 'Примерный вес', 'Тип металла', 'Проба',
            'Покрытие', 'Тип вставки', 'Форма вставки', 'Количество', 'Цвет', 'Вес',
            'Цветность', 'Чистота', 'Огранка', 'Размеры вставки', 'Тип вставки 2', 'Форма вставки 2',
            'Количество 2', 'Цвет 2', 'Вес 2', 'Цветность 2', 'Чистота 2',
            'Огранка 2', 'Размеры вставки 2', 'Ширина кольца', 'Ширина', 'Высота',
            'Длина', 'Номер страницы', 'Цена со скидкой', 'Цена без скидки']]

In [671]:
data.head()

,Артикул,Бренд,Коллекция,Для кого,Примерный вес,Тип металла,Проба,Покрытие,Тип вставки,Форма вставки,...,Чистота 2,Огранка 2,Размеры вставки 2,Ширина кольца,Ширина,Высота,Длина,Номер страницы,Цена со скидкой,Цена без скидки
0,1012159-3,SOKOLOV,NaN,Для женщин,1.66 г,Белое золото,585.0,Родирование,Бриллиант (природный),Круг,...,6.0,А,NaN,NaN,NaN,NaN,NaN,1,38250,84990
1,1011493,SOKOLOV,NaN,Для женщин,1.35 г,Белое золото,585.0,Родирование,Бриллиант (природный),Круг,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,35100,77990
2,1012584-3,SOKOLOV,NaN,Для женщин,0.9 г,Белое золото,585.0,Родирование,Бриллиант (природный),Круг,...,NaN,NaN,NaN,2 мм,2 мм,1 мм,NaN,1,7990,7990
3,1011116,SOKOLOV,NaN,Для женщин,2.28 г,Белое золото,585.0,NaN,Бриллиант (природный),Круг,...,3.0,А,NaN,2 мм,5 мм,4 мм,5 мм,1,99000,219990
4,1011545-3,SOKOLOV,NaN,Для женщин,1.31 г,Белое золото,585.0,Родирование,Бриллиант (природный),Круг,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,56250,124990


Получилось 33 колонки, однако мы не разобрались с упомянутыми в начале колонками, где много пустых значений. Поставим порог в 65% от всех колец и удалим колонки, в которых пустых значений больше порога.

In [672]:
nan_col = []
for i in range(len(data.isna().sum())):
    if data.isna().sum()[i] / data.shape[0] > 0.65:
        nan_col.append(data.isna().sum().index[i])
nan_col

['Коллекция',
 'Цветность',
 'Чистота',
 'Огранка',
 'Цветность 2',
 'Чистота 2',
 'Огранка 2',
 'Размеры вставки 2',
 'Длина']

Так как у `Размеры вставки 2` имеют много пустых значений, будет логичным удалить и столбец `Размеры вставки`, чтобы каждый тип вставки характеризовался одними признаками.

Помимо этого `Бренд` в данном датасете один - SOKOLOV, поэтому его тоже можно дропнуть.

`Артикул` также не несет никакой информации, его необходимо удалить, но сделаем это в самом конце, так как по артикулу можно смотреть, какие конкретно кольца чем-либо выделяются.

In [673]:
nan_col.extend(['Размеры вставки', 'Бренд'])
nan_col
data.drop(nan_col, axis=1, inplace=True)
data.head()

,Артикул,Для кого,Примерный вес,Тип металла,Проба,Покрытие,Тип вставки,Форма вставки,Количество,Цвет,...,Форма вставки 2,Количество 2,Цвет 2,Вес 2,Ширина кольца,Ширина,Высота,Номер страницы,Цена со скидкой,Цена без скидки
0,1012159-3,Для женщин,1.66 г,Белое золото,585.0,Родирование,Бриллиант (природный),Круг,6 шт,Бесцветный,...,Круг,1 шт,Бесцветный,0.02 карат,NaN,NaN,NaN,1,38250,84990
1,1011493,Для женщин,1.35 г,Белое золото,585.0,Родирование,Бриллиант (природный),Круг,1 шт,Бесцветный,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,35100,77990
2,1012584-3,Для женщин,0.9 г,Белое золото,585.0,Родирование,Бриллиант (природный),Круг,23 шт,Бесцветный,...,NaN,NaN,NaN,NaN,2 мм,2 мм,1 мм,1,7990,7990
3,1011116,Для женщин,2.28 г,Белое золото,585.0,NaN,Бриллиант (природный),Круг,1 шт,Бесцветный,...,Круг,12 шт,Бесцветный,0.119 карат,2 мм,5 мм,4 мм,1,99000,219990
4,1011545-3,Для женщин,1.31 г,Белое золото,585.0,Родирование,Бриллиант (природный),Круг,57 шт,Бесцветный,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,56250,124990


> ### Шаг 3: преобразование типов данных

В датасете присутствуют количественные признаки, однако они все в формате `str` (кроме пробы), и многие с указанием единиц измерения. Нужно привести их к типу данных `float`.

In [674]:
data.dtypes

Артикул             object
Для кого            object
Примерный вес       object
Тип металла         object
Проба              float64
Покрытие            object
Тип вставки         object
Форма вставки       object
Количество          object
Цвет                object
Вес                 object
Тип вставки 2       object
Форма вставки 2     object
Количество 2        object
Цвет 2              object
Вес 2               object
Ширина кольца       object
Ширина              object
Высота              object
Номер страницы       int64
Цена со скидкой      int64
Цена без скидки      int64
dtype: object

In [675]:
num_col = ['Примерный вес', 'Количество', 'Вес', 'Количество 2', 'Вес 2', 'Ширина', 'Высота']
for col in num_col:
    data[col] = data[col].apply(lambda x: float(x.split()[0]) if type(x) == str else x)
data.head()

,Артикул,Для кого,Примерный вес,Тип металла,Проба,Покрытие,Тип вставки,Форма вставки,Количество,Цвет,...,Форма вставки 2,Количество 2,Цвет 2,Вес 2,Ширина кольца,Ширина,Высота,Номер страницы,Цена со скидкой,Цена без скидки
0,1012159-3,Для женщин,1.66,Белое золото,585.0,Родирование,Бриллиант (природный),Круг,6.0,Бесцветный,...,Круг,1.0,Бесцветный,0.020,NaN,NaN,NaN,1,38250,84990
1,1011493,Для женщин,1.35,Белое золото,585.0,Родирование,Бриллиант (природный),Круг,1.0,Бесцветный,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,35100,77990
2,1012584-3,Для женщин,0.90,Белое золото,585.0,Родирование,Бриллиант (природный),Круг,23.0,Бесцветный,...,NaN,NaN,NaN,NaN,2 мм,2.0,1.0,1,7990,7990
3,1011116,Для женщин,2.28,Белое золото,585.0,NaN,Бриллиант (природный),Круг,1.0,Бесцветный,...,Круг,12.0,Бесцветный,0.119,2 мм,5.0,4.0,1,99000,219990
4,1011545-3,Для женщин,1.31,Белое золото,585.0,Родирование,Бриллиант (природный),Круг,57.0,Бесцветный,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,56250,124990


Отдельно преобразуем формат столбца `Ширина кольца`, так как в нем есть значение `'1. 6 мм'`, которое отформатируем отдельно.

In [676]:
data['Ширина кольца'].unique()

array([nan, '2 мм', '1 мм', '5 мм', '3 мм', '4 мм', '6 мм', '7 мм',
       '8 мм', '11 мм', '9 мм', '12 мм', '15 мм', '10 мм', '13 мм',
       '17 мм', '20 мм', '14 мм', '1, 6 мм'], dtype=object)

In [677]:
data['Ширина кольца'] = data['Ширина кольца'].apply(lambda x: float(x.split()[0]) if (type(x) == str) and (x != '1, 6 мм') else x)
data.loc[data['Ширина кольца'] == '1, 6 мм', 'Ширина кольца'] = 1.6
data.dtypes

Артикул             object
Для кого            object
Примерный вес      float64
Тип металла         object
Проба              float64
Покрытие            object
Тип вставки         object
Форма вставки       object
Количество         float64
Цвет                object
Вес                float64
Тип вставки 2       object
Форма вставки 2     object
Количество 2       float64
Цвет 2              object
Вес 2              float64
Ширина кольца       object
Ширина             float64
Высота             float64
Номер страницы       int64
Цена со скидкой      int64
Цена без скидки      int64
dtype: object

> ### Шаг 4: Заполняем пустые значения


Вернемся к `nan`, их у нас много, поэтому необходимо их по-умному заполнить.

In [678]:
data.isna().sum()

Артикул               0
Для кого             24
Примерный вес         9
Тип металла           0
Проба                43
Покрытие           2985
Тип вставки           5
Форма вставки      1111
Количество          983
Цвет               1122
Вес                 987
Тип вставки 2      5086
Форма вставки 2    5194
Количество 2       5086
Цвет 2             5208
Вес 2              5088
Ширина кольца      3194
Ширина             4082
Высота             4094
Номер страницы        0
Цена со скидкой       0
Цена без скидки       0
dtype: int64

1. В столбцах `Для кого` и `Покрытие` заполним пропуски словом `Неизвестно`, так как нам никак не определить, для кого предназначается это кольцо по каким-либо признакам (быть может по отзывам/фото, но это слишком сложно), как и не определить покрытие без точного указания данного признака. Далее при кодировании данных признаков при помощи OHE можно будет удалить столбцы с названием `Неизвестно`...

In [679]:
print(data['Для кого'].unique())
print()
print(data['Покрытие'].unique())

['Для женщин' 'Для женщин, Для мужчин, Унисекс' 'Для мужчин' 'Для детей'
 nan 'Для женщин, Для мужчин, Для детей']

['Родирование' nan 'Алмазная грань,  Родирование' 'Эмаль' 'Алмазная грань'
 'Золочение' 'Золочение,  Эмаль' 'Родирование,  Эмаль'
 'Золочение,  Родирование' 'Гравировка,  Родирование' 'Гравировка'
 'Золочение,  Родирование,  Эмаль' 'Гравировка,  Золочение'
 'Лазерная обработка,  Родирование' 'Чернение'
 'Алмазная грань,  Родирование,  Эмаль']


In [680]:
data[['Для кого', 'Покрытие']] = data[['Для кого', 'Покрытие']].fillna('Неизвестно')

2. Пропуски в столбцах `Примерный вес`, `Ширина`, `Ширина кольца`, `Высота` заменим средними.

In [681]:
columns_nan = ['Примерный вес', 'Ширина кольца', 'Ширина', 'Высота']
data[columns_nan] = data[columns_nan].fillna(data[columns_nan].mean())

3. `Проба` - заменим средними в зависимости от типа металла

In [682]:
# пупупу

4. Далее переходим к вставкам. В столбце `Тип вставки` всего 5 пропущенных значений, посмотрим на них.

Все эти кольца, если посмотреть на сайте, имеют специфические вставки: керамические, хромдиопси, поэтому их можно убрать из выборки (как выбросы).

In [683]:
data[data['Тип вставки'].isna()]

,Артикул,Для кого,Примерный вес,Тип металла,Проба,Покрытие,Тип вставки,Форма вставки,Количество,Цвет,...,Форма вставки 2,Количество 2,Цвет 2,Вес 2,Ширина кольца,Ширина,Высота,Номер страницы,Цена со скидкой,Цена без скидки
3471,94013741,Для женщин,0.17,Родированное серебро,925.0,Неизвестно,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,5.000000,4.000000,61,3820,8490
4096,717010,Для женщин,1.60,Красное золото,585.0,Неизвестно,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,7.605504,3.755826,71,18000,59990
4666,716396,Для женщин,2.59,Красное золото,585.0,Неизвестно,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,7.605504,3.755826,81,32850,72990
4669,716405,Для женщин,1.23,Красное золото,585.0,Неизвестно,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,7.605504,3.755826,81,21600,47990
4962,94013807,Для женщин,0.35,Родированное серебро,925.0,Неизвестно,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,6.000000,4.000000,86,3820,8490


In [684]:
data.drop(data[data['Тип вставки'].isna()].index, inplace=True)

Теперь в столбце `Тип вставки` остались сами типы вставок, а также значение `Без вставок`. У колец без камней можно заменить все категориальные признаки вставок на значение `Без вставок` (это `Форма вставки` и `Цвет`), а количественные - на `0` (это `Количество`, `Вес`).

In [685]:
data.columns

Index(['Артикул', 'Для кого', 'Примерный вес', 'Тип металла', 'Проба',
       'Покрытие', 'Тип вставки', 'Форма вставки', 'Количество', 'Цвет', 'Вес',
       'Тип вставки 2', 'Форма вставки 2', 'Количество 2', 'Цвет 2', 'Вес 2',
       'Ширина кольца', 'Ширина', 'Высота', 'Номер страницы',
       'Цена со скидкой', 'Цена без скидки'],
      dtype='object')

In [686]:
cat_vstavki =['Форма вставки', 'Цвет']
num_vstavki = ['Количество', 'Вес']
data.loc[data['Тип вставки'] == 'Без вставок', cat_vstavki] = data.loc[data['Тип вставки'] == 'Без вставок', cat_vstavki].fillna('Без вставок')
data.loc[data['Тип вставки'] == 'Без вставок', num_vstavki] = data.loc[data['Тип вставки'] == 'Без вставок', num_vstavki].fillna(0)

Заменим также все пропущенные значения в `Тип вставки 2` на `Без вставок`, а все признаки второй вставки по аналогии с первой.

In [687]:
data['Тип вставки 2'] = data['Тип вставки 2'].fillna('Без вставок')
cat_vstavki2 =['Форма вставки 2', 'Цвет 2']
num_vstavki2 = ['Количество 2', 'Вес 2']
data.loc[data['Тип вставки 2'] == 'Без вставок', cat_vstavki2] = data.loc[data['Тип вставки 2'] == 'Без вставок', cat_vstavki2].fillna('Без вставок')
data.loc[data['Тип вставки 2'] == 'Без вставок', num_vstavki2] = data.loc[data['Тип вставки 2'] == 'Без вставок', num_vstavki2].fillna(0)

Оставшиеся пустые значения заменим на `Неизвестно` у качественных признаков, и на среднее у количественных

In [688]:
cat_ost = ['Форма вставки', 'Цвет', 'Форма вставки 2', 'Цвет 2']
num_ost = ['Вес', 'Вес 2']

data[cat_ost] = data[cat_ost].fillna('Неизвестно')
data[num_ost] = data[num_ost].fillna(data[num_ost].mean())
data.isna().sum()

Артикул             0
Для кого            0
Примерный вес       0
Тип металла         0
Проба              43
Покрытие            0
Тип вставки         0
Форма вставки       0
Количество          0
Цвет                0
Вес                 0
Тип вставки 2       0
Форма вставки 2     0
Количество 2        0
Цвет 2              0
Вес 2               0
Ширина кольца       0
Ширина              0
Высота              0
Номер страницы      0
Цена со скидкой     0
Цена без скидки     0
dtype: int64